In [1]:
from transformers import pipeline
import pickle
import json
import pandas as pd


import tensorflow as tf
import tensorflow_hub as hub
#import tensorflow_text as text

/Users/emilychen/opt/anaconda3/envs/fbads/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import requests
from bs4 import BeautifulSoup
import re

In [3]:
classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)
2022-03-07 17:14:09.646269: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-03-07 17:14:09.666868: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7facae8370f0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-03-07 17:14:09.666888: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-03-07 17:14:09.772524: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
A

In [4]:
f = open("Ads10000_election", "rb")
ads = pickle.load(f)
f.close()

In [5]:
df = pd.json_normalize(ads)
df.fillna(0, inplace = True)
df = df[df.get('ad_creative_link_descriptions') != 0]
df['ad_creative_link_descriptions'] = df.get('ad_creative_link_descriptions').apply(lambda x: x[0])
print(df.shape)
df.head()

(7416, 16)


,ad_creation_time,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,ad_delivery_start_time,ad_delivery_stop_time,id,estimated_audience_size.lower_bound,impressions.lower_bound,impressions.upper_bound,spend.lower_bound,spend.upper_bound,delivery_by_region,demographic_distribution,estimated_audience_size.upper_bound
0,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-04,711213506429873,1000001,0,999,0,99,0,0,0
1,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-01,127634835910715,1000001,0,999,0,99,0,0,0
2,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-04,3821823704574505,1000001,0,999,0,99,0,0,0
3,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-03,745901799329077,1000001,0,999,0,99,0,0,0
4,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-03,172031037686911,1000001,0,999,0,99,0,0,0


In [6]:
def find_label(x):
    for class_dict in classifier(x[:512]):
        return(class_dict['label'])

In [7]:
df['sentiment'] = df.get('ad_creative_link_descriptions').apply(find_label)
df.head()

,ad_creation_time,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,ad_delivery_start_time,ad_delivery_stop_time,id,estimated_audience_size.lower_bound,impressions.lower_bound,impressions.upper_bound,spend.lower_bound,spend.upper_bound,delivery_by_region,demographic_distribution,estimated_audience_size.upper_bound,sentiment
0,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-04,711213506429873,1000001,0,999,0,99,0,0,0,POSITIVE
1,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-01,127634835910715,1000001,0,999,0,99,0,0,0,POSITIVE
2,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-04,3821823704574505,1000001,0,999,0,99,0,0,0,POSITIVE
3,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-03,745901799329077,1000001,0,999,0,99,0,0,0,POSITIVE
4,2021-02-01,[County closes downtown San Diego vaccine site...,[https://apps.apple.com/us/app/smartnews-local...,‎SmartNews is the award-winning news app downl...,[News for San Diego County],2021-02-01,2021-02-03,172031037686911,1000001,0,999,0,99,0,0,0,POSITIVE


In [8]:
neg_sent = df[df.get('sentiment') == 'NEGATIVE']
print(neg_sent.shape)
neg_sent.head()

(2511, 17)


,ad_creation_time,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,ad_delivery_start_time,ad_delivery_stop_time,id,estimated_audience_size.lower_bound,impressions.lower_bound,impressions.upper_bound,spend.lower_bound,spend.upper_bound,delivery_by_region,demographic_distribution,estimated_audience_size.upper_bound,sentiment
11,2021-02-01,[Tennessee to Begin COVID-19 Vaccination of Ad...,[covid19.tn.gov],When the vaccine supply is limited it is neces...,[Tennessee Vaccine Phases | TN COVID-19 Hub],2021-02-01,2021-02-03,836656640224916,1000001,1000,1999,0,99,"[{'percentage': '0.002398', 'region': 'Colorad...","[{'percentage': '0.000806', 'age': '18-24', 'g...",0,NEGATIVE
28,2021-02-01,[La vacuna contra el COVID-19 estará disponibl...,[www1.nyc.gov],Obtén información,[Acerca de la vacuna contra el COVID-19],2021-02-01,2021-02-14,138514571438503,1000001,500000,599999,2500,2999,"[{'percentage': '1', 'region': 'New York'}]","[{'percentage': '5.0E-6', 'age': 'Unknown', 'g...",0,NEGATIVE
34,2021-02-01,[💉 “Operation Warp Speed has delivered two Cad...,[the-rotation.com],Up to 50 million Americans could receive Covid...,[A Rocky Rollout: Covid-19 Vaccine Distributio...,2021-02-01,2021-03-08,865641954230328,500001,200000,249999,400,499,"[{'percentage': '0.020282', 'region': 'Alabama...","[{'percentage': '0.000173', 'age': '65+', 'gen...",1000000,NEGATIVE
49,2021-02-01,"[DeSoto County News: January Almost Gone, & Co...",[newsbreak.com],Boxes containing the Moderna COVID-19 vaccine ...,[Breaking news from DeSoto County!],2021-02-01,2021-02-01,158635512535229,100001,0,999,0,99,"[{'percentage': '0.957816', 'region': 'Mississ...","[{'percentage': '0.012165', 'age': '45-54', 'g...",500000,NEGATIVE
53,2021-02-01,[Johnson County News: Here are the Walmart sto...,[newsbreak.com],Walmart has started vaccinating Texans with th...,[Breaking news from Johnson County!],2021-02-01,2021-02-01,426807908535217,100001,0,999,0,99,"[{'percentage': '1', 'region': 'Texas'}]","[{'percentage': '0.005814', 'age': '45-54', 'g...",500000,NEGATIVE


In [9]:
neg_sent['impressions.lower_bound'] = neg_sent.get('impressions.lower_bound').apply(lambda x: int(x))
neg_sent['impressions.upper_bound'] = neg_sent.get('impressions.upper_bound').apply(lambda x: int(x))
neg_sent['average_est_impressions'] = (neg_sent['impressions.lower_bound'] + neg_sent['impressions.upper_bound'])/2
neg_sent.head()

/Users/emilychen/.local/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/emilychen/.local/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/emilychen/.local/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value 

,ad_creation_time,ad_creative_bodies,ad_creative_link_captions,ad_creative_link_descriptions,ad_creative_link_titles,ad_delivery_start_time,ad_delivery_stop_time,id,estimated_audience_size.lower_bound,impressions.lower_bound,impressions.upper_bound,spend.lower_bound,spend.upper_bound,delivery_by_region,demographic_distribution,estimated_audience_size.upper_bound,sentiment,average_est_impressions
11,2021-02-01,[Tennessee to Begin COVID-19 Vaccination of Ad...,[covid19.tn.gov],When the vaccine supply is limited it is neces...,[Tennessee Vaccine Phases | TN COVID-19 Hub],2021-02-01,2021-02-03,836656640224916,1000001,1000,1999,0,99,"[{'percentage': '0.002398', 'region': 'Colorad...","[{'percentage': '0.000806', 'age': '18-24', 'g...",0,NEGATIVE,1499.5
28,2021-02-01,[La vacuna contra el COVID-19 estará disponibl...,[www1.nyc.gov],Obtén información,[Acerca de la vacuna contra el COVID-19],2021-02-01,2021-02-14,138514571438503,1000001,500000,599999,2500,2999,"[{'percentage': '1', 'region': 'New York'}]","[{'percentage': '5.0E-6', 'age': 'Unknown', 'g...",0,NEGATIVE,549999.5
34,2021-02-01,[💉 “Operation Warp Speed has delivered two Cad...,[the-rotation.com],Up to 50 million Americans could receive Covid...,[A Rocky Rollout: Covid-19 Vaccine Distributio...,2021-02-01,2021-03-08,865641954230328,500001,200000,249999,400,499,"[{'percentage': '0.020282', 'region': 'Alabama...","[{'percentage': '0.000173', 'age': '65+', 'gen...",1000000,NEGATIVE,224999.5
49,2021-02-01,"[DeSoto County News: January Almost Gone, & Co...",[newsbreak.com],Boxes containing the Moderna COVID-19 vaccine ...,[Breaking news from DeSoto County!],2021-02-01,2021-02-01,158635512535229,100001,0,999,0,99,"[{'percentage': '0.957816', 'region': 'Mississ...","[{'percentage': '0.012165', 'age': '45-54', 'g...",500000,NEGATIVE,499.5
53,2021-02-01,[Johnson County News: Here are the Walmart sto...,[newsbreak.com],Walmart has started vaccinating Texans with th...,[Breaking news from Johnson County!],2021-02-01,2021-02-01,426807908535217,100001,0,999,0,99,"[{'percentage': '1', 'region': 'Texas'}]","[{'percentage': '0.005814', 'age': '45-54', 'g...",500000,NEGATIVE,499.5


In [10]:
region_df = pd.DataFrame()

In [11]:
for index, row in neg_sent.iterrows():
    if row['delivery_by_region'] == 0:
        continue
    else:
        d = pd.DataFrame(row['delivery_by_region'])
        d['percentage'] = d.get('percentage').apply(float)
        d['impressions_per_state'] = d.get('percentage') * row['average_est_impressions']
        region_df = region_df.append(d)
        
region_df.head()

,percentage,region,impressions_per_state
0,0.002398,Colorado,3.595801
1,0.025580,Ohio,38.357210
2,0.018385,Minnesota,27.568307
3,0.008793,Mississippi,13.185104
4,0.011191,Missouri,16.780904


In [12]:
region_df = region_df.groupby('region').mean().reset_index()
region_df.head()

,region,percentage,impressions_per_state
0,Abia State,0.000506,3.794747
1,Abkhazia,0.000253,1.897374
2,Abruzzo,0.000475,5.937263
3,Abu Dhabi,0.011201,139.380649
4,Adana Province,0.000253,1.897374


In [13]:
us_states = ['Alabama', 'Alaska', 'Arizona', 'Arkansas', 'California', 'Colorado', 'Connecticut', 'Washington, District of Columbia', 'Delaware', 'Florida', 'Georgia', 'Hawaii', 'Idaho', 'Illinois', 'Indiana', 'Iowa', 'Kansas', 'Kentucky', 'Louisiana', 'Maine', 'Maryland', 'Massachusetts', 'Michigan', 'Minnesota', 'Mississippi', 'Missouri', 'Montana', 'Nebraska', 'Nevada', 'New Hampshire', 'New Jersey', 'New Mexico', 'New York', 'North Carolina', 'North Dakota', 'Ohio', 'Oklahoma', 'Oregon', 'Pennsylvania', 'Rhode Island', 'South Carolina', 'South Dakota', 'Tennessee', 'Texas', 'Utah', 'Vermont', 'Virginia', 'Washington', 'West Virginia', 'Wisconsin', 'Wyoming']

In [14]:
region_df = region_df[region_df.get('region').isin(us_states) == True]
region_df.get('region').iloc[47] = 'District of Columbia'
region_df.sort_values(by = 'region', inplace = True)

In [15]:
import altair as alt
import geopandas as gpd
import json
from vega_datasets import data
pop = data.population_engineers_hurricanes()
region_df['id'] = list(pop.get('id'))[:-1]
region_df

,region,percentage,impressions_per_state,id
17,Alabama,0.035121,682.298340,1
18,Alaska,0.003885,135.196590,2
37,Arizona,0.198760,3793.505665,4
38,Arkansas,0.016927,396.149228,5
102,California,0.151670,3408.622939,6
153,Colorado,0.044799,1632.620693,8
157,Connecticut,0.021885,387.972514,9
168,Delaware,0.017750,712.712790,10
671,District of Columbia,0.037075,3718.867848,11
201,Florida,0.146395,3609.411751,12


In [16]:
alt.renderers.enable('default')

RendererRegistry.enable('default')

In [17]:
states = alt.topo_feature(data.us_10m.url, feature='states')

alt.Chart(states).mark_geoshape(
    fill='lightgray',
    stroke='white'
).project('albersUsa').properties(
    width=500,
    height=300
)

alt.Chart(...)

In [18]:
from vega_datasets import data

states = alt.topo_feature(data.us_10m.url, 'states')

variable_list = ['percentage', 'impressions_per_state']

# Creating configs for color,selection,hovering
hover = alt.selection(type='single', on='mouseover', nearest=True,
                      fields=['x', 'y'])

plot = alt.Chart(states).mark_geoshape().encode(
    color='impressions_per_state:Q'
).transform_lookup(
    lookup='id',
    from_=alt.LookupData(region_df, 'id', list(region_df.columns))
).properties(
    width=500,
    height=300
).project(
    type='albersUsa'
)

outline = alt.Chart(states).mark_geoshape(stroke='black', fillOpacity=0).project(
    type='albersUsa'
).properties(
    width=500,
    height=300
)

alt.layer(plot,outline)

alt.LayerChart(...)